In [1]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
# from datasets import Dataset
# from tqdm import tqdm
# import torch
# import json


In [2]:
# # llm_judge.py
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# from model_loader.config import get_model_loader 

# class LLMJudge:
#     def __init__(self, judge_llm_name: str, max_seq_length: int = 512):
#         print(f"판단 모델 로딩 중: {judge_llm_name} (get_model_loader 사용)...")
        
#         judge_quantization_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_compute_dtype=torch.bfloat16,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4"
#         )

#         try:
#             judge_loader_instance = get_model_loader(
#                 provider="local",
#                 model_path=judge_llm_name,
#                 quantization_config=judge_quantization_config,
#                 torch_dtype=torch.bfloat16
#             )
#             self.tokenizer = judge_loader_instance.tokenizer
#             self.model = judge_loader_instance.model
#         except Exception as e:
#             print(f"판단 모델 로더에서 오류 발생: {e}")
#             raise

#         if self.tokenizer.pad_token is None:
#             self.tokenizer.pad_token = self.tokenizer.eos_token
#             self.model.config.pad_token_id = self.tokenizer.eos_token_id
            
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         self.model.to(self.device)
#         self.model.eval() 
#         self.max_seq_length = max_seq_length

#     def _create_judge_prompt(self, question: str, context: str, generated_answer: str, true_answer: str) -> str:
#         return (
#             f"다음은 질문, 문서, 생성된 답변, 그리고 정답입니다.\n"
#             f"생성된 답변이 정답과 의미적으로 동일하거나 정확한지 판단하여 '정답' 또는 '오답'으로만 답변해주세요.\n\n"
#             f"질문: {question}\n"
#             f"문서: {context}\n"
#             f"생성된 답변: {generated_answer}\n"
#             f"정답: {true_answer}\n\n"
#             f"판단: "
#         )

#     def is_correct(self, question: str, context: str, generated_answer: str, true_answer: str) -> bool:
#         judge_prompt = self._create_judge_prompt(question, context, generated_answer, true_answer)
        
#         inputs = self.tokenizer(judge_prompt, return_tensors="pt", 
#                                 max_length=self.max_seq_length, truncation=True).to(self.device)
        
#         with torch.no_grad():
#             outputs = self.model.generate(
#                 **inputs,
#                 max_new_tokens=10, 
#                 num_beams=1,
#                 do_sample=False,
#                 temperature=0.0,
#                 pad_token_id=self.tokenizer.pad_token_id,
#                 eos_token_id=self.tokenizer.eos_token_id
#             )
        
#         judge_result = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], 
#                                               skip_special_tokens=True).strip().lower()

#         if "오답" in judge_result or "틀림" in judge_result or "incorrect" in judge_result:
#             return False
#         return True

In [3]:
# # slearn_llm.py
# import json
# from transformers import TrainingArguments, Trainer, BitsAndBytesConfig
# from datasets import Dataset
# import torch
# from tqdm import tqdm
# import os 

# from model_loader.config import get_model_loader, generation_loader


# def slearn_llm_training(
#     model_name: str = "./model/LLM/deepseek-qwen-bllossom-32b",
#     data_path: str = "./data/qna_data.json",
#     output_dir: str = "./model",
#     max_seq_length: int = 512,
#     batch_size: int = 4,
#     learning_rate: float = 2e-5,
#     num_train_epochs: int = 3,
#     eval_steps: int = 500,
#     save_steps: int = 500,
#     judge_llm_name: str = "google/gemma-2b"
# ):
#     print(f"학습 모델 로딩 중: {model_name} (model_loader.config.generation_loader 사용)...")
#     tokenizer = generation_loader.tokenizer
#     model = generation_loader.model

#     if tokenizer.pad_token is None:
#         tokenizer.pad_token = tokenizer.eos_token
#         model.config.pad_token_id = tokenizer.eos_token_id
    
#     model.to("cuda" if torch.cuda.is_available() else "cpu") 

#     llm_judge = LLMJudge(judge_llm_name=judge_llm_name, max_seq_length=max_seq_length)

#     print(f"데이터셋 로딩 중: {data_path}")
#     with open(data_path, 'r', encoding="utf-8") as f:
#         qna_data = json.load(f)

#     log_dir = os.path.join(output_dir, "logs")
#     os.makedirs(log_dir, exist_ok=True) 
#     evaluation_log_path = os.path.join(log_dir, "evaluation_log.txt")

#     generated_qna_data = []
#     model.eval()
#     print("초기 LLM 답변 생성 중...")
#     with open(evaluation_log_path, 'w', encoding='utf-8') as log_file: 
#         log_file.write("=== SLearnLLM Initial Evaluation Log ===\n\n")
        
#         for idx, item in enumerate(tqdm(qna_data, desc="프리딕션 진행 중")):
#             question = item["question"]
#             context = item["context"]
#             true_answer = item["answer"]

#             input_text_prompt = f"질문 : {question}\n문서 : {context}\n답변 : " 
#             inputs = tokenizer(input_text_prompt, return_tensors="pt", max_length=max_seq_length, truncation=True).to(model.device)

#             with torch.no_grad():
#                 outputs = model.generate(
#                     **inputs,
#                     max_new_tokens=128,
#                     num_beams=1,
#                     do_sample=False,
#                     temperature=0.0,
#                     pad_token_id=tokenizer.pad_token_id,
#                     eos_token_id=tokenizer.eos_token_id
#                 )

#             generated_answer = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

#             is_correct_prediction = llm_judge.is_correct(
#                 question=question,
#                 context=context,
#                 generated_answer=generated_answer,
#                 true_answer=true_answer
#             )
            
#             generated_qna_data.append({
#                 "question": question,
#                 "context": context,
#                 "true_answer": true_answer,
#                 "generated_answer": generated_answer,
#                 "input_text_prompt": input_text_prompt,
#                 "is_correct_judged": is_correct_prediction
#             })

#             log_file.write(f"--- QA Pair {idx + 1} ---\n")
#             log_file.write(f"질문: {question}\n")
#             log_file.write(f"문서: {context}\n")
#             log_file.write(f"실제 정답: {true_answer}\n")
#             log_file.write(f"생성된 답변: {generated_answer}\n")
#             log_file.write(f"판단 결과: {'정답' if is_correct_prediction else '오답'}\n")
#             log_file.write("-" * 30 + "\n\n")

#     print(f"초기 평가 로그가 '{evaluation_log_path}'에 저장되었습니다.")

#     incorrect_qna_pairs = []
#     print("판단된 QA 쌍 필터링 중...")
#     for item in tqdm(generated_qna_data, desc="필터링 진행 중"):
#         if not item["is_correct_judged"]:
#             incorrect_qna_pairs.append({
#                 "question": item["question"],
#                 "context": item["context"],
#                 "answer": item["true_answer"],
#                 "input_text": item["input_text_prompt"],
#                 "output_text": item["true_answer"]
#             })

#     print(f"총 {len(qna_data)}개 중 {len(incorrect_qna_pairs)}개의 오답 쌍 발견 (LLM 판단 기준).")
#     if not incorrect_qna_pairs:
#         print("모든 QA 쌍을 정확하게 예측했다고 판단되었습니다. 추가 학습이 필요 없습니다.")
#         return
    
#     print("오답 쌍으로 학습 데이터셋 생성 중...")
#     filtered_data = []
#     for item in incorrect_qna_pairs:
#         filtered_data.append({
#             "input_text": item["input_text"],
#             "output_text": item["output_text"]
#         })

#     dataset = Dataset.from_list(filtered_data)

#     def tokenize_function(examples):
#         full_text = [
#             f"{examples['input_text'][i]}{examples['output_text'][i]}{tokenizer.eos_token}"
#             for i in range(len(examples['input_text']))
#         ]
        
#         tokenized_inputs = tokenizer(
#             full_text,
#             max_length=max_seq_length,
#             truncation=True,
#             padding="max_length"
#         )

#         labels = []
#         for i in range(len(examples['input_text'])):
#             input_prompt_len = len(tokenizer(examples['input_text'][i], truncation=False)['input_ids'])
            
#             label = list(tokenized_inputs['input_ids'][i])
#             for j in range(input_prompt_len):
#                 label[j] = -100
#             labels.append(label)
        
#         tokenized_inputs["labels"] = labels
#         return tokenized_inputs

#     tokenized_dataset = dataset.map(
#         tokenize_function,
#         batched=True,
#         remove_columns=dataset.column_names
#     )

#     print("필터링된 오답 쌍으로 학습 LLM 재학습 시작...")
#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         per_device_train_batch_size=batch_size,
#         learning_rate=learning_rate,
#         num_train_epochs=num_train_epochs,
#         logging_dir=f"{output_dir}/logs",
#         logging_steps=100,
#         save_strategy="steps",
#         save_steps=save_steps,
#         evaluation_strategy="steps",
#         eval_steps=eval_steps,
#         load_best_model_at_end=True,
#         metric_for_best_model="loss",
#         greater_is_better=False,
#         fp16=True if torch.cuda.is_available() else False,
#         gradient_accumulation_steps=1,
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_dataset,
#         tokenizer=tokenizer,
#     )

#     trainer.train()
#     print("학습 완료! 모델 저장 중...")
#     model.save_pretrained(output_dir)
#     tokenizer.save_pretrained(output_dir)
#     print(f"학습된 모델이 {output_dir}에 저장되었습니다.")

# if __name__ == "__main__":
#     slearn_llm_training(
#         model_name="./model/LLM/deepseek-qwen-bllossom-32b",
#         data_path="./data/qna_data.json",
#         output_dir="./model/slearn_llm_finetuned_llm_judge_modular",
#         max_seq_length=512,
#         batch_size=2,
#         learning_rate=2e-5,
#         num_train_epochs=3,
#         eval_steps=100,
#         save_steps=100,
#         judge_llm_name="google/gemma-2b" 
#     )

In [ ]:
# if __name__ == "__main__":
#     slearn_llm_training(
#         model_name="./model/LLM/deepseek-qwen-bllossom-32b",
#         data_path="./data/qna_data.json",
#         output_dir="./model/slearn_llm_finetuned_llm_judge_modular",
#         max_seq_length=512,
#         batch_size=2,
#         learning_rate=2e-5,
#         num_train_epochs=3,
#         eval_steps=100,
#         save_steps=100,
#         judge_llm_name="./model/LLM/deepseek-qwen-bllossom-32b" 
#     )


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import Dataset
import json
from tqdm import tqdm
import os

_global_models = {}

class LLMJudge:
    def __init__(self, judge_llm_name: str, max_seq_length: int = 512):
        
        config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )
        if judge_llm_name not in _global_models:
            self.tokenizer = AutoTokenizer.from_pretrained(judge_llm_name)
            self.model = AutoModelForCausalLM.from_pretrained(judge_llm_name, torch_dtype=torch.bfloat16, quantization_config=config, device_map="auto")
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.tokenizer.eos_token_id
            
            self.model.eval()
            
            self.device = self.model.device
            _global_models[judge_llm_name] = {"tokenizer": self.tokenizer, "model": self.model, "device": self.device}
        else:
            loaded_data = _global_models[judge_llm_name]
            self.tokenizer = loaded_data["tokenizer"]
            self.model = loaded_data["model"]
            self.device = loaded_data["device"]
            
        self.max_seq_length = max_seq_length

    def _create_judge_prompt(self, question: str, generated_answer: str, true_answer: str) -> str:
        return (
            f"다음은 [질문], [생성된 답변], 그리고 [기준 정답]입니다.\n"
            f"[기준 정답]을 기준으로 [생성된 답변]이 정답/오답인지 여부를 판단하세요. \n"
            f"[생성된 답변]이 **실제로 맞는 답변이더라도 [기준 정답]과 다르면 '오답'으로 판단**해야 합니다.\n\n"
            f"[질문]: {question}\n"
            f"[생성된 답변]: {generated_answer}\n"
            f"[기준 정답]: {true_answer}\n\n" 
            f"판단: "
        )

    def is_correct(self, question: str, generated_answer: str, true_answer: str) -> bool:
        judge_prompt = self._create_judge_prompt(question, generated_answer, true_answer)
        
        inputs = self.tokenizer(judge_prompt, return_tensors="pt", 
                                max_length=self.max_seq_length, truncation=True).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=10,
                num_beams=1,
                do_sample=False,
                temperature=0.0,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
            
        judge_result = self.tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], 
                                             skip_special_tokens=True).strip().lower()

        if "오답" in judge_result:
            return False
        return True

def slearn_llm_training_with_llm_judge_modular(
    model_name: str = "./model/LLM/deepseek-qwen-bllossom-32b",
    data_path: str = "./data/qna_data.json",
    output_dir: str = "./model",
    max_seq_length: int = 512,
    batch_size: int = 4,
    learning_rate: float = 2e-5,
    num_train_epochs: int = 3,
    eval_steps: int = 500,
    save_steps: int = 500,
    judge_llm_name: str = "google/gemma-2b"
):
    global _global_models # 전역 변수 사용 명시

    config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )
    
    if model_name not in _global_models:
        model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                     torch_dtype=torch.bfloat16, 
                                                     device_map="auto",
                                                     quantization_config=config)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = tokenizer.eos_token_id
        _global_models[model_name] = {"tokenizer": tokenizer, "model": model, "device": model.device}
    else:
        loaded_data = _global_models[model_name]
        tokenizer = loaded_data["tokenizer"]
        model = loaded_data["model"]
    
    llm_judge = LLMJudge(judge_llm_name=judge_llm_name, max_seq_length=max_seq_length)

    with open(data_path, 'r', encoding='utf-8') as f:
        qna_data = json.load(f)

    generated_qna_data = []
    for item in tqdm(qna_data, desc="답변 생성 중"):
        question = item['question']
        context = item['context']
        true_answer = item['answer']

        input_text_prompt = f"질문 : {question}\n답변 : "
        inputs = tokenizer(input_text_prompt, return_tensors="pt", max_length=max_seq_length, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                num_beams=1,
                do_sample=True,
                temperature=0.3,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        generated_answer = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
        
        generated_qna_data.append({
            "question": question,
            "context": context,
            "true_answer": true_answer,
            "generated_answer": generated_answer,
            "input_text_prompt": input_text_prompt
        })

    incorrect_qna_pairs = []
    
    output_results_dir = "./data/slearn"
    os.makedirs(output_results_dir, exist_ok=True)

    for idx, item in enumerate(tqdm(generated_qna_data, desc="판단 진행 중")):
        is_correct_prediction = llm_judge.is_correct(
            question=item['question'],
            # context=item['context'],
            generated_answer=item['generated_answer'],
            true_answer=item['true_answer']
        )
        
        if not is_correct_prediction:
            incorrect_qna_pairs.append({
                "question": item['question'],
                "context": item['context'],
                "answer": item['true_answer'],
                "input_text": item['input_text_prompt'],
                "output_text": item['true_answer']
            })
        
        result_status = "정답" if is_correct_prediction else "오답"
        file_name = os.path.join(output_results_dir, f"slearn_result_{idx+1:05d}.txt")
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(f"<질문>: {item['question']}\n\n")
            f.write(f"<문서>: {item['context']}\n\n")
            f.write(f"<생성된 답변>: {item['generated_answer']}\n\n")
            f.write(f"<기준 정답>: {item['true_answer']}\n\n")
            f.write(f"<판단 결과>: {result_status}\n")

    print(f"총 {len(qna_data)}개의 QA 쌍 중 {len(incorrect_qna_pairs)}개가 오답으로 판단되었습니다.")
    
    if not incorrect_qna_pairs:
        return

    filtered_data = []
    for item in incorrect_qna_pairs:
        filtered_data.append({
            "input_text": item["input_text"],
            "output_text": item["output_text"]
        })

    dataset = Dataset.from_list(filtered_data)

    def tokenize_function(examples):
        full_text = [
            f"{examples['input_text'][i]}{examples['output_text'][i]}{tokenizer.eos_token}"
            for i in range(len(examples['input_text']))
        ]
        
        tokenized_inputs = tokenizer(
            full_text,
            max_length=max_seq_length,
            truncation=True,
            padding="max_length"
        )

        labels = []
        for i in range(len(examples['input_text'])):
            input_prompt_len = len(tokenizer(examples['input_text'][i], truncation=False)['input_ids'])
            
            label = list(tokenized_inputs['input_ids'][i])
            for j in range(input_prompt_len):
                label[j] = -100
            labels.append(label)
        
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset.column_names
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        logging_dir=f"{output_dir}/logs",
        logging_steps=100,
        save_strategy="steps",
        save_steps=save_steps,
        eval_strategy="steps",
        eval_steps=eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        fp16=True if torch.cuda.is_available() else False,
        gradient_accumulation_steps=1,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()
    
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

if __name__ == "__main__":
    slearn_llm_training_with_llm_judge_modular(
        model_name="./model/LLM/deepseek-qwen-bllossom-32b",
        data_path="./data/qna_data.json",
        output_dir="./model/LLM/slearn_llm_finetuned_llm_judge_modular",
        max_seq_length=512,
        batch_size=2,
        learning_rate=2e-5,
        num_train_epochs=3,
        eval_steps=100,
        save_steps=100,
        judge_llm_name="./model/LLM/EEVE"
    )

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

판단 진행 중:   0%|          | 0/20 [00:00<?, ?it/s]/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
판단 진행 중: 100%|██████████| 20/20 [00:12<00:00,  1.60it/s]

총 20개의 QA 쌍 중 5개가 오답으로 판단되었습니다.


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/tmp/ipykernel_1016023/1477319817.py:240: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.STEPS but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

: 